# 决策树代码实现

In [44]:
import numpy as np
import operator

In [45]:
def loaddata():
    dataSet = [[0,0,0,0,'no'],
               [0,0,0,1,'no'],
               [0,1,0,1,'yes'],
               [0,1,1,0,'yes'],
               [0,0,0,0,'no'],
               [1,0,0,0,'no'],
               [1,0,0,1,'no'],
               [1,1,1,1,'yes'],
               [1,0,1,2,'yes'],
               [1,0,1,2,'yes'],
               [2,0,1,2,'yes'],
               [2,0,1,1,'yes'],
               [2,1,0,1,'yes'],
               [2,1,0,2,'yes'],
               [2,0,0,0,'no']]
    featureNames = ['age','job','hosuse','credit','class']
    return dataSet, featureNames

### 计算数据集的熵
<img src="images/02.png" width="200" align="left"/>

In [46]:
from math import log2
from collections import Counter
def entropy(dataSet):
    """
    计算信息熵
    :param y: 标签列表
    :return: 信息熵
    """
    if(isinstance(dataSet[0],list)):
        dataSet = [example[-1] for example in dataSet]
    counter = Counter(dataSet)

    total_count = len(dataSet)
    entropy = 0.0
    for count in counter.values():
        p = count / total_count
        entropy -= p * log2(p)
    return entropy

# 划分数据集

In [47]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet



### 选择最好的特征
<img src="images/03.png" width="400" align="left"/>

In [48]:
def chooseBestFeature(dataSet):
    n = len(dataSet[0]) - 1
    baseEntropy = entropy(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(n):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * entropy(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature



# 类别的投票表决

In [49]:
def classVote(classList):
    """
    计算classList中出现次数最多的类别
    """
    classCount = {}
    for vote in classList:
        if vote not in classCount:
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=lambda x: x[1], reverse=True)
    return sortedClassCount[0][0]

In [50]:
classList = ['yes','no','yes','no','yes','no','yes','no','yes','no']
classVote(classList)

'yes'

# 递归训练决策树

In [51]:
def trainTree(dataSet,featureNames):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return classVote(classList)
    bestFeat = chooseBestFeature(dataSet)
    bestFeatName = featureNames[bestFeat]
    myTree = {bestFeatName:{}}
    #del(featureNames[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subFeatureName = featureNames[:]
        subLabels = splitDataSet(dataSet,bestFeat,value)
        myTree[bestFeatName][value] = trainTree(subLabels,subFeatureName)
    return myTree

In [52]:
dataSet,featureNames = loaddata()
myTree = trainTree(dataSet,featureNames)
print(myTree)

{'hosuse': {0: {'job': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


In [55]:
def predict(inputTree,featureNames,testVec):    
    firstStr = list(inputTree.keys())[0]
    print("inputTree.keys():",firstStr)
    secondDict = inputTree[firstStr]
    featIndex = featureNames.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat,dict):
        classLabel = predict(valueOfFeat,featureNames,testVec)
    else:
        classLabel = valueOfFeat
    return classLabel

In [58]:
from math import log2
from collections import Counter
class DecisionTree:
    def __init__(self):
        self.tree = None
        self.featureNames = []

    def entropy(self,dataSet):
        """
        计算信息熵
        :param y: 标签列表
        :return: 信息熵
        """
        if(isinstance(dataSet[0],list)):
            dataSet = [example[-1] for example in dataSet]
        counter = Counter(dataSet)

        total_count = len(dataSet)
        entropy = 0.0
        for count in counter.values():
            p = count / total_count
            entropy -= p * log2(p)
        return entropy
    
    def splitDataSet(self,dataSet, axis, value):
        retDataSet = []
        for featVec in dataSet:
            if featVec[axis] == value:
                reducedFeatVec = featVec[:axis]
                reducedFeatVec.extend(featVec[axis+1:])
                retDataSet.append(reducedFeatVec)
        return retDataSet
    
    def chooseBestFeature(self,dataSet):
        n = len(dataSet[0]) - 1
        baseEntropy = self.entropy(dataSet)
        bestInfoGain = 0.0
        bestFeature = -1
        for i in range(n):
            featList = [example[i] for example in dataSet]
            uniqueVals = set(featList)
            newEntropy = 0.0
            for value in uniqueVals:
                subDataSet = self.splitDataSet(dataSet, i, value)
                prob = len(subDataSet) / float(len(dataSet))
                newEntropy += prob * self.entropy(subDataSet)
            infoGain = baseEntropy - newEntropy
            if (infoGain > bestInfoGain):
                bestInfoGain = infoGain
                bestFeature = i
        return bestFeature    

    def classVote(self,classList):
        """
        计算classList中出现次数最多的类别
        """
        classCount = {}
        for vote in classList:
            if vote not in classCount:
                classCount[vote] = 0
            classCount[vote] += 1
        sortedClassCount = sorted(classCount.items(), key=lambda x: x[1], reverse=True)
        return sortedClassCount[0][0]
    
    def trainTree(self,dataSet,featureNames):
        classList = [example[-1] for example in dataSet]
        if classList.count(classList[0]) == len(classList):
            return classList[0]
        if len(dataSet[0]) == 1:
            return self.classVote(classList)
        bestFeat = self.chooseBestFeature(dataSet)
        bestFeatName = featureNames[bestFeat]
        myTree = {bestFeatName:{}}
        #del(featureNames[bestFeat])
        featValues = [example[bestFeat] for example in dataSet]
        uniqueVals = set(featValues)
        for value in uniqueVals:
            subFeatureName = featureNames[:]
            subLabels = self.splitDataSet(dataSet,bestFeat,value)
            myTree[bestFeatName][value] = self.trainTree(subLabels,subFeatureName)
        return myTree    
    
    def fit(self,dataSet,featureNames):
        self.featureNames = featureNames
        self.tree = self.trainTree(dataSet,featureNames)

    def predict(self,testVec):
        inputTree = self.tree
        featureNames = self.featureNames
        firstStr = list(inputTree.keys())[0]
        print("inputTree.keys():",firstStr)
        secondDict = inputTree[firstStr]
        featIndex = featureNames.index(firstStr)
        key = testVec[featIndex]
        valueOfFeat = secondDict[key]
        if isinstance(valueOfFeat,dict):
            classLabel = predict(valueOfFeat,featureNames,testVec)
        else:
            classLabel = valueOfFeat
        return classLabel




In [59]:
model = DecisionTree()
dataSet,featureNames = loaddata()
model.fit(dataSet,featureNames)
model.predict([1,1,0,1])

inputTree.keys(): hosuse
inputTree.keys(): job


'yes'